# 模型训练完整流程

In [2]:
# 介绍一个函数，在分类问题中常用
# torch.argmax
#     input,                   输入张量
#     dim=None,                指定沿哪个维度找最大值的位置（默认：展平成1维后求最大值索引）
#     keepdim=False            是否保持原始维度（True返回shape中保留该dim维)

# 示例：
    # x = torch.tensor([[1, 5, 2],
    #                   [7, 3, 9]])
    # # 按行取最大值的索引（dim=1）
    # row_max_idx = torch.argmax(x, dim=1)
    # print(row_max_idx)  # tensor([1, 2]) -> 每行最大值索引位置

    # # 按列取最大值的索引（dim=0）
    # col_max_idx = torch.argmax(x, dim=0)
    # print(col_max_idx)  # tensor([1, 0, 1]) -> 每列最大值索引位置

In [3]:
import torch
import torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict
from Model_save import LeNet_5

In [4]:
# 1. 准备数据集
train_data = torchvision.datasets.CIFAR10("./data/CIFAR10", train=True, download=True,
                                           transform=torchvision.transforms.ToTensor())
val_data = torchvision.datasets.CIFAR10("./data/CIFAR10", train = False, download=True, transform = torchvision.transforms.ToTensor())

# 2. 查看数据集大小
train_data_size = len(train_data)
val_data_size = len(val_data)
print("训练数据集长度为：{}".format(train_data_size))
print("测试数据集长度为：{}".format(val_data_size))

# 3. 创建数据加载器
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=64)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
训练数据集长度为：50000
测试数据集长度为：10000


In [5]:
# 4. 创建模型，网络见Model_save.py
model = LeNet_5()

# 5. 定义损失函数
loss_fn = nn.CrossEntropyLoss()

# 6. 定义优化器
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [6]:
# 设置训练网络的一些参数
#记录训练的次数
total_train_step = 0
#记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10
#绘制训练图像
writer = SummaryWriter("./logs/17_Model_training")

In [8]:
# 7. 训练模型
for i in range(epoch):
    print("---------------------第{}轮训练开始---------------------".format(i + 1))

    # 训练步骤
    model.train()
    for data in train_dataloader:
        img, target = data
        output = model(img)
        loss = loss_fn(output, target)

        # 优化器梯度清零
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

        # 查看训练结果
        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, loss：{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)


    model.eval()
    # 每一轮训练结束之后在测试集上验证模型结果，对模型进行评估，在测试集上不对魔心进行调优
    total_test_loss = 0
    with torch.no_grad():
        right_sum = 0
        for data in val_dataloader:
            imgs, targets = data
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            outputs = torch.argmax(outputs, dim=1)

            right_sum += torch.sum(outputs == targets).sum()
        print("整体测试集上的loss：{}".format(total_test_loss))
        writer.add_scalar("test_loss", total_test_loss, total_test_step)
        print("整体测试集上的正确率：{}".format(right_sum / 10000))
        writer.add_scalar("test_accuracy", right_sum / 10000, total_test_step)
        total_test_step += 1

    # 8. 保存模型
    # 方式一：
    # torch.save(model, "./models/LeNet_5/LeNet_5_No_{}_loss_{}.pth".format(epoch, total_test_step))
    # 方式二：（推荐）
    torch.save(model.state_dict(), "./models/LeNet_5/LeNet_5_No_{}_loss_{}.pth".format(epoch, total_test_step))
    print("模型已保存")

writer.close()


---------------------第1轮训练开始---------------------
训练次数：800, loss：1.8553754091262817
训练次数：900, loss：1.8357659578323364
训练次数：1000, loss：1.9563456773757935
训练次数：1100, loss：1.960581660270691
训练次数：1200, loss：1.707315444946289
训练次数：1300, loss：1.671413779258728
训练次数：1400, loss：1.7597821950912476
训练次数：1500, loss：1.801198959350586
整体测试集上的loss：295.43255150318146
整体测试集上的正确率：0.32359999418258667
模型已保存
---------------------第2轮训练开始---------------------
训练次数：1600, loss：1.721817135810852
训练次数：1700, loss：1.6426852941513062
训练次数：1800, loss：1.9466661214828491
训练次数：1900, loss：1.7322067022323608
训练次数：2000, loss：1.9288573265075684
训练次数：2100, loss：1.5295467376708984
训练次数：2200, loss：1.455535888671875
训练次数：2300, loss：1.7748297452926636
整体测试集上的loss：270.7079870700836
整体测试集上的正确率：0.3716999888420105
模型已保存
---------------------第3轮训练开始---------------------
训练次数：2400, loss：1.700143814086914
训练次数：2500, loss：1.3359973430633545


KeyboardInterrupt: 